In [2]:
import torch

# Get the PyTorch version
torch_version = torch.__version__

# Check if CUDA is available and get its version
cuda_version = torch.version.cuda if torch.cuda.is_available() else "CUDA not available"

torch_version, cuda_version


('2.4.1', '11.8')

In [4]:
#conda install -c dglteam dgl
#pip install plotly
#pip install xgi
#pip install python-louvain

In [5]:
import dgl

### Load edge and node data

In [29]:
import pandas as pd 
SVI_edges = pd.read_csv('data/graph_building/london_10_landmarks/hetergenous_graph/hetergenous_graph_edges.csv')
SVI_edges

,source,target
0,virtual_100009526,qloOWVZHq3GZEcvENulBtg
1,smI5NVr3YGCHhZ2xGLeVgA,rcT4e98neeJLvDqEKP0Ubw
2,rcT4e98neeJLvDqEKP0Ubw,"virtual_1013566017,1185063017,1185063018,10135..."
3,rcT4e98neeJLvDqEKP0Ubw,B0u0gr1_OECuu45S493L5Q
4,rcT4e98neeJLvDqEKP0Ubw,ohjUTeWeswYYozzbsayGXg
...,...,...
107752,QQ0cc2_O2s0o4YHot-z--Q,RO7W7YDGgLYy8uQecXFbIA
107753,RO7W7YDGgLYy8uQecXFbIA,pN2dUbPJ2ohIp8La6cGzRw
107754,pN2dUbPJ2ohIp8La6cGzRw,Tho45agomox1_gCz41zvwA
107755,RraQcNkpxyrO4_KGO73RvA,fcoTfNB7iI7NLMjOd42ZGg


In [30]:
#import geopandas as gpd
import numpy as np
#SVI_landmark_edges = gpd.read_file('data/graph_building/london_10_landmarks/hetergenous_graph/street_level_nodes_with_attributes.gpkg')
SVI_landmark_edges = pd.read_csv('data/graph_building/london_10_landmarks/hetergenous_graph/street_level_nodes_with_attributes.csv')

#SVI_landmark_edges .loc[SVI_landmark_edges['shortest_distance']==np.inf, 'shortest_distance'] = 100
SVI_landmark_edges.columns = ['Unnamed: 0', 'node_id', 'latitude', 'longitude', 'visibility',
       'landmark', 'geometry', 'distance_to_landmark', 'proximity',
       'building_count', 'area_density', 'shortest_distance']
SVI_landmark_edges

,Unnamed: 0,node_id,latitude,longitude,visibility,landmark,geometry,distance_to_landmark,proximity,building_count,area_density,shortest_distance
0,0,virtual_100009526,182064.117851,530293.357058,-1,big_ben,POINT (530293.3570582942 182064.11785114277),2426.003086,0,77,44.9,7.384657
1,1,virtual_100009526,182064.117851,530293.357058,-1,gherkin,POINT (530293.3570582942 182064.11785114277),3117.292647,0,77,44.9,7.384657
2,2,virtual_100009526,182064.117851,530293.357058,-1,london_eye,POINT (530293.3570582942 182064.11785114277),2152.807401,0,77,44.9,7.384657
3,3,virtual_100009526,182064.117851,530293.357058,-1,st_paul,POINT (530293.3570582942 182064.11785114277),1989.965447,0,77,44.9,7.384657
4,4,virtual_100009526,182064.117851,530293.357058,-1,tower_bridge,POINT (530293.3570582942 182064.11785114277),3813.838434,0,77,44.9,7.384657
...,...,...,...,...,...,...,...,...,...,...,...,...
354615,354615,E21yLcuPQFfru_q7MsM86Q,182020.488628,532201.348947,0,the_shard,POINT (532201.3489466362 182020.48862825328),2001.178334,0,21,37.1,4.060284
354616,354616,E21yLcuPQFfru_q7MsM86Q,182020.488628,532201.348947,0,london_tower,POINT (532201.3489466362 182020.48862825328),2041.510859,0,21,37.1,4.060284
354617,354617,E21yLcuPQFfru_q7MsM86Q,182020.488628,532201.348947,0,tate_modern,POINT (532201.3489466362 182020.48862825328),1544.880168,0,21,37.1,4.060284
354618,354618,E21yLcuPQFfru_q7MsM86Q,182020.488628,532201.348947,0,belfast,POINT (532201.3489466362 182020.48862825328),1966.944767,0,21,37.1,4.060284


In [31]:
landmark_df = pd.read_csv('data/graph_building/london_10_landmarks/hetergenous_graph/landmark_points.csv')
landmark_df 

,Unnamed: 0,latitude,longitude,name,height,area,building_count,area_density,geometry
0,0,51.500677,-0.124592,big_ben,96,144,30.0,25.4,POINT (530270.542225827 179638.22204615263)
1,1,51.514426,-0.080343,gherkin,180,2511,118.0,57.5,POINT (533301.564583585 181246.682267542)
2,2,51.503257,-0.119686,london_eye,135,1145,21.0,15.5,POINT (530603.6545632128 179933.79037252557)
3,3,51.513725,-0.098357,st_paul,111,7920,66.0,47.3,POINT (532053.6920602581 181136.1183036333)
4,4,51.505595,-0.075539,tower_bridge,65,2080,40.0,8.7,POINT (533660.6764366097 180273.44093802886)
5,5,51.504625,-0.086527,the_shard,310,3728,55.0,62.1,POINT (532900.937647139 180145.57786330278)
6,6,51.511261,-0.083534,walkie_talkie,160,3721,NaN,NaN,POINT (533087.2739131616 180885.3646662379)
7,7,51.507896,-0.099348,tate_modern,99,13169,NaN,NaN,POINT (532001.5274018036 180488.5858621825)
8,8,51.508086,-0.076079,london_tower,27,1174,NaN,NaN,POINT (533614.111322055 180546.76237876498)
9,9,51.506660,-0.081383,belfast,20,3039,NaN,NaN,POINT (533265.2996709987 180366.13661782764)


In [32]:
### here we have both virtual and real nodes, 9495,
### the real nodes are only about
len(SVI_landmark_edges.node_id.unique())

35462

In [33]:
#SVI_landmark_edges.to_csv('data/hetergenous_graph/20241215_london_the_shard_3000m_real_virtual_SVI_nodes.csv') 

## Build hetergenous graph

In [721]:
import dgl
import torch
from collections import defaultdict, Counter
from tqdm import tqdm

# Define heterogeneous graph
hetero_graph = dgl.heterograph({
    ('svi_location', 'visibility', 'landmark'): [],     # visibility: SVI → Landmark
    ('landmark', 'proximity', 'svi_location'): [],      # proximity: Landmark → SVI
    ('svi_location', 'SVI_connectivity', 'svi_location'): [],  # undirected edge handling
    ('landmark', 'seen_together', 'hyperedge'): [],
    ('hyperedge', 'connects', 'landmark'): [],
    ('landmark', 'see_from_location', 'landmark'): [],  # maintain directionality
})

# Define landmark and svi_location nodes
landmark_ids = {q: idx for idx, q in enumerate(SVI_landmark_edges.landmark.unique())}
hetero_graph.add_nodes(len(landmark_ids), ntype='landmark')

svi_node_ids = {s: idx for idx, s in enumerate(SVI_landmark_edges.node_id.unique())}
hetero_graph.add_nodes(len(svi_node_ids), ntype='svi_location')

# Add visibility edges (SVI → Landmark) and remove duplicates
visibility_edges = list(set([
    (svi_node_ids[row['node_id']], landmark_ids[row['landmark']])
    for _, row in tqdm(SVI_landmark_edges.iterrows(), total=SVI_landmark_edges.shape[0], desc="Adding visibility edges")
    if row['visibility'] > 0
]))

hetero_graph.add_edges(*zip(*visibility_edges), etype='visibility')

# Add proximity edges (Landmark → SVI) and remove duplicates
proximity_edges = list(set([
    (landmark_ids[row['landmark']], svi_node_ids[row['node_id']])
    for _, row in tqdm(SVI_landmark_edges.iterrows(), total=SVI_landmark_edges.shape[0], desc="Adding proximity edges")
    if row['proximity'] == 1
]))

hetero_graph.add_edges(*zip(*proximity_edges), etype='proximity')

# Add undirected SVI_connectivity edges
connectivity_edges = [
    (svi_node_ids[row['source']], svi_node_ids[row['target']])
    for _, row in tqdm(SVI_edges.iterrows(), total=SVI_edges.shape[0], desc="Adding connectivity edges")
]

# Normalize edges to ensure (src, dst) and (dst, src) are treated as the same edge
all_connectivity_edges = list(set([
    (min(src, dst), max(src, dst)) for src, dst in connectivity_edges
]))

hetero_graph.add_edges(*zip(*all_connectivity_edges), etype='SVI_connectivity')

# Add Hyperedges, remove duplicate landmarks
svi_to_landmarks = defaultdict(list)
for _, row in tqdm(SVI_landmark_edges.iterrows(), total=SVI_landmark_edges.shape[0], desc="Mapping SVI to landmarks"):
    if row['visibility'] > 0:
        svi_to_landmarks[row['node_id']].append(landmark_ids[row['landmark']])

hyperedge_id_start = 0
for svi_node, landmarks in tqdm(svi_to_landmarks.items(), desc="Adding hyperedges"):
    unique_landmarks = list(set(landmarks))  # remove duplicates
    if len(unique_landmarks) > 1:
        hyperedge_id = hyperedge_id_start
        hyperedge_id_start += 1

        hetero_graph.add_nodes(1, ntype='hyperedge')
        hetero_graph.add_edges(unique_landmarks, [hyperedge_id] * len(unique_landmarks), etype='seen_together')
        hetero_graph.add_edges([hyperedge_id] * len(unique_landmarks), unique_landmarks, etype='connects')

# Calculate see_from_location edges
see_from_location_counter = Counter()
for _, row in tqdm(SVI_landmark_edges.iterrows(), total=SVI_landmark_edges.shape[0], desc="Counting proximity-visibility edges"):
    if row['proximity'] == 1:
        landmark1 = landmark_ids[row['landmark']]
        svi_node = svi_node_ids[row['node_id']]
        other_landmarks = [
            landmark_ids[other_row['landmark']]
            for _, other_row in SVI_landmark_edges[SVI_landmark_edges['node_id'] == row['node_id']].iterrows()
            if other_row['visibility'] > 0 and other_row['landmark'] != row['landmark']
        ]
        for landmark2 in other_landmarks:
            see_from_location_counter[(landmark1, landmark2)] += 1

# Add see_from_location edges
if see_from_location_counter:
    src_nodes, dst_nodes, weights = zip(*[(u, v, w) for (u, v), w in see_from_location_counter.items()])
    hetero_graph.add_edges(src_nodes, dst_nodes, etype='see_from_location')
    hetero_graph.edges['see_from_location'].data['weight'] = torch.tensor(weights, dtype=torch.float32)

# Add Landmark and SVI features
landmark_attributes = {col: [] for col in ['latitude', 'longitude', 'height', 'area', 'building_count', 'area_density']}
for landmark_name, landmark_id in tqdm(landmark_ids.items(), desc="Adding landmark attributes"):
    attributes = landmark_df.loc[landmark_df['name'] == landmark_name]
    for col in landmark_attributes.keys():
        landmark_attributes[col].append(attributes.iloc[0][col] if not attributes.empty else 0.0)

for col, values in landmark_attributes.items():
    hetero_graph.nodes['landmark'].data[col] = torch.tensor(values, dtype=torch.float32)

svi_attributes = {col: [] for col in ['latitude', 'longitude', 'building_count', 'area_density', 'shortest_distance', 'distance_to_landmark']}
for svi_node_id in tqdm(svi_node_ids, desc="Adding SVI attributes"):
    attributes = SVI_landmark_edges.loc[SVI_landmark_edges['node_id'] == svi_node_id]
    for col in svi_attributes.keys():
        svi_attributes[col].append(attributes.iloc[0][col] if not attributes.empty else 0.0)

for col, values in svi_attributes.items():
    hetero_graph.nodes['svi_location'].data[col] = torch.tensor(values, dtype=torch.float32)

# Print graph construction completion message
print("Graph construction completed.")
print("All edges in the graph are DIRECTED unless specified as undirected with reverse edges.")

Adding landmark attributes: 100%|██████████████████████████████████████████████████████| 10/10 [00:00<00:00, 80.49it/s]


Graph construction completed.
All edges in the graph are DIRECTED unless specified as undirected with reverse edges.


In [722]:
isolated_nodes = hetero_graph.in_degrees(etype='visibility') == 0
print(f"Number of isolated SVI nodes: {isolated_nodes.sum().item()}")


Number of isolated SVI nodes: 0


In [ ]:
import torch

def check_node_attributes(hetero_graph, node_type):
    print(f"\nChecking attributes for node type: {node_type}")
    for key, tensor in hetero_graph.nodes[node_type].data.items():
        if torch.isnan(tensor).any():
            print(f"WARNING: NaN values found in '{key}' attribute of {node_type} nodes")
        if torch.isinf(tensor).any():
            print(f"WARNING: Inf values found in '{key}' attribute of {node_type} nodes")
    print("No NaN or Inf values found in", node_type)

# Check landmark attributes
check_node_attributes(hetero_graph, 'landmark')

# Check SVI location attributes
check_node_attributes(hetero_graph, 'svi_location')


In [724]:
import torch

def clean_tensor(tensor):
    """ Replace NaN and Inf values in a PyTorch tensor with 0. """
    tensor = torch.where(torch.isnan(tensor), torch.tensor(0.0, dtype=torch.float32), tensor)
    tensor = torch.where(torch.isinf(tensor), torch.tensor(0.0, dtype=torch.float32), tensor)
    return tensor
# Check & fix NaN/Inf in landmark attributes
for col in landmark_attributes.keys():
    tensor = torch.tensor(landmark_attributes[col], dtype=torch.float32)
    cleaned_tensor = clean_tensor(tensor)
    hetero_graph.nodes['landmark'].data[col] = cleaned_tensor

# Check & fix NaN/Inf in SVI location attributes
for col in svi_attributes.keys():
    tensor = torch.tensor(svi_attributes[col], dtype=torch.float32)
    cleaned_tensor = clean_tensor(tensor)
    hetero_graph.nodes['svi_location'].data[col] = cleaned_tensor


In [726]:
if 'feat' not in hetero_graph.nodes['hyperedge'].data:
    hyperedge_feat_dim = 4
    num_hyperedges = hetero_graph.num_nodes('hyperedge')
    hetero_graph.nodes['hyperedge'].data['feat'] = torch.zeros((num_hyperedges, hyperedge_feat_dim))
    
if 'weight' not in hetero_graph.edges['visibility'].data:
    hetero_graph.edges['visibility'].data['weight'] = torch.ones(hetero_graph.num_edges('visibility'))
if 'weight' not in hetero_graph.edges['proximity'].data:
    hetero_graph.edges['proximity'].data['weight'] = torch.ones(hetero_graph.num_edges('proximity'))
    
if 'weight' not in hetero_graph.edges['SVI_connectivity'].data:
    hetero_graph.edges['SVI_connectivity'].data['weight'] = torch.ones(hetero_graph.num_edges('SVI_connectivity'))

if 'weight' not in hetero_graph.edges['seen_together'].data:
    hetero_graph.edges['seen_together'].data['weight'] = torch.ones(hetero_graph.num_edges('seen_together'))
if 'weight' not in hetero_graph.edges['connects'].data:
    hetero_graph.edges['connects'].data['weight'] = torch.ones(hetero_graph.num_edges('connects'))

#if 'weight' not in hetero_graph.edges['see_from_location'].data:
#    hetero_graph.edges['see_from_location'].data['weight'] = torch.ones(hetero_graph.num_edges('see_from_location'))


In [727]:
# Assume `hetero_graph` is your DGL graph
file_path = 'data/graph_building/london_10_landmarks/hetergenous_graph/0123_10_landmarks_6_edges_graph.dgl'

# Save the graph
dgl.save_graphs(file_path, hetero_graph)

print(f"Graph saved to {file_path}")

Graph saved to data/graph_building/london_10_landmarks/hetergenous_graph/0123_10_landmarks_6_edges_graph.dgl


In [728]:
print(hetero_graph)

Graph(num_nodes={'hyperedge': 206, 'landmark': 10, 'svi_location': 35462},
      num_edges={('hyperedge', 'connects', 'landmark'): 466, ('landmark', 'proximity', 'svi_location'): 1347, ('landmark', 'see_from_location', 'landmark'): 24, ('landmark', 'seen_together', 'hyperedge'): 466, ('svi_location', 'SVI_connectivity', 'svi_location'): 107757, ('svi_location', 'visibility', 'landmark'): 1619},
      metagraph=[('hyperedge', 'landmark', 'connects'), ('landmark', 'svi_location', 'proximity'), ('landmark', 'landmark', 'see_from_location'), ('landmark', 'hyperedge', 'seen_together'), ('svi_location', 'svi_location', 'SVI_connectivity'), ('svi_location', 'landmark', 'visibility')])
